In [7]:
import pandas as pd
import os

In [8]:
def read_ply(file_name):
    vertices = []
    with open(file_name, 'r') as file:
        lines = file.readlines()
        header_end = 0
        for i, line in enumerate(lines):
            if line.strip() == 'end_header':
                header_end = i + 1
                break
        vertex_lines = lines[header_end:]
        for line in vertex_lines:
            parts = line.split()
            x, y, z = map(float, parts[:3])  # x, y, z coordinates
            vertices.append((x, y, z))
    return vertices

In [9]:
def write_ply(file_name, vertices, block_id):
    # Start writing the header
    with open(file_name, 'w') as file:
        file.write('ply\n')
        file.write('format ascii 1.0\n')
        file.write(f'element vertex {len(vertices)}\n')
        file.write('property float x\n')
        file.write('property float y\n')
        file.write('property float z\n')
        file.write('property float scalar_intensity\n')
        file.write('property float scalar_blockid\n')
        file.write('end_header\n')

        # Write the vertices with the new properties
        for (x, y, z) in vertices:
            file.write(f'{x} {y} {z} 1 {block_id}\n')

In [10]:
csv_files = ["partition_summary_Campus_BlockBa.csv",  "partition_summary_Campus_BlockBb.csv",  "partition_summary_Campus_BlockBc.csv"]
for csv_file in csv_files:
    suffix = csv_file[-5:-4]
    df = pd.read_csv(csv_file)
    grouped_files = []
    current_group = []
    current_size = 0
    
    # Block ID counter
    block_id_counter = 1
    
    # Iterate through the CSV rows to group by file size under 1 GB
    for _, row in df.iterrows():
        block_id = row['Block_ID']
        file_name = row['File_Name']
        size_mb = row['Size_MB']
    
        if current_size + size_mb <= 1024:  # If the cumulative size is less than 1GB
            current_group.append(file_name)
            current_size += size_mb
        else:
            # Process the current group of files
            grouped_files.append(current_group)
            
            # Reset the current group and size for the next one
            current_group = [file_name]
            current_size = size_mb
    
    # Don't forget to add the last group
    if current_group:
        grouped_files.append(current_group)
    
    # Step 5: Process each group of files
    output_dir = 'grouped_ply_files'
    os.makedirs(output_dir, exist_ok=True)
    
    for group in grouped_files:
        all_vertices = []
        
        # Read all the PLY files in the current group and accumulate vertices
        for file_name in group:
            ply_vertices = read_ply(file_name)
            all_vertices.extend(ply_vertices)
    
        # Remove duplicates based on x, y, z
        unique_vertices = list(set(all_vertices))  # Remove duplicate vertices based on x, y, z
    
        # Step 6: Write the unique vertices to a new PLY file with a new Block ID
        new_file_name = os.path.join(output_dir, f"Campus_BlockB{suffix}_group_{block_id_counter}.ply")
        write_ply(new_file_name, unique_vertices, block_id_counter)
        
        # Increment the block ID counter for the next file group
        block_id_counter += 1
    
    print(f"Grouped PLY files have been written to the directory: {output_dir}")

Grouped PLY files have been written to the directory: grouped_ply_files
Grouped PLY files have been written to the directory: grouped_ply_files
Grouped PLY files have been written to the directory: grouped_ply_files
